## 1. Dataset Class 선언
- Data: Spam베이스라는 57개의 feature을 통해서 spam이메일인지 아닌지 여부를 확인하는 데이터셋, github 참고 

In [1]:
# 코렙용
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
# torch dataloader
from torch.utils.data import DataLoader, Dataset
import torch
#train_test_split
class SpamDataloader(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]



class Spam:

    def __init__(self):
        self.train_dir = 'src/dataset/spambase' + '/spambase.data'

        pass

    def create_data(self):
        X = pd.read_csv(self.train_dir, sep=',', header=None)
        #data.dropna(axis=1, how='all', inplace=True)
        X=X.values
        Y=X[:,-1]
        X=X[:,:-1]
        Y = Y.astype(int)
        from sklearn.model_selection import train_test_split
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
        X_train = torch.tensor(X_train).float()
        Y_train = torch.tensor(Y_train).float()
        X_test = torch.tensor(X_test).float()
        Y_test = torch.tensor(Y_test).float()
        Spam_train = SpamDataloader(X_train, Y_train)
        Spam_test = SpamDataloader(X_test, Y_test)
        return Spam_train, Spam_test


/tmp/ipykernel_1586713/2198860499.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Simple Neural Network 선언 이때 생성자에서 하이퍼파라미터를 받을 수 있도록 처리 

In [3]:
import torch.nn as nn
class SimpleNN(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

        
    def forward(self, x):
        # x: batch_size x 10
        x = self.linear1(x)
        #question
        # batchsize x 5
        x = self.linear2(x)
        # x: batch_size x 1
        x = self.sigmoid(x)
        # x: batch_size x 1
        
        return x
    

## Trainer Class 선언: Training과 Testing 둘다 진행가능

In [4]:

class Trainer():

    def __init__(self, model, dataloader, optimizer, loss_fn,device):
        # model, dataloader, optimizer, loss_fn
        self.model = model
        self.dataloader = dataloader
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.device = device
        #self.model=self.model.to(device) #  move the model to the device
    
    def train(self, num_epochs):
        for epoch in range(num_epochs):
            for (x, y) in self.dataloader:
                x=x.to(self.device)
                y=y.to(self.device)
                #print(x.device)
                y_pred = self.model(x)
                y_pred = y_pred.squeeze()
                loss = self.loss_fn(y_pred, y)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()


            print("Epoch: {}, Loss: {}".format(epoch, loss.item()))

    def test(self,test_dataloader):
        with torch.no_grad():
            self.model.eval()
            for (x, y) in test_dataloader:
                x=x.to(self.device)
                y=y.to(self.device)
                y_pred = self.model(x)
                y_pred = y_pred.squeeze()
                loss = self.loss_fn(y_pred, y)
                print("Test Loss: {}".format(loss.item()))

## 실제 실행  데이터 처리-> 모델 정의 -> 트레이터 클래스 정의 -> 테스트 처리 

In [5]:
spam = Spam()
Spam_train,Spam_test = spam.create_data() # X: train, Y: train_label, X: test

# Create a dataloader
train_dataloader = DataLoader(Spam_train, batch_size=32, shuffle=True)
test_dataloader = DataLoader(Spam_test, batch_size=3000, shuffle=True) # batch_size = 3000을 한 이유는 test data 전체를 한번에 testing 하기 위함임. 


# Create a model
model = SimpleNN(input_dim=57, hidden_dim=10,output_dim= 1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Create an optimizer
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.0001)
# Create a loss function
loss_fn = torch.nn.BCELoss()
model=model.to(device)
# Create a trainer
trainer = Trainer(model, train_dataloader, optimizer, loss_fn,device)
# Train the model
trainer.train(num_epochs=50)
# Test the model
trainer.test(test_dataloader)


Epoch: 0, Loss: 16.89145278930664
Epoch: 1, Loss: 4.5887346267700195
Epoch: 2, Loss: 0.728247880935669
Epoch: 3, Loss: 0.6062618494033813
Epoch: 4, Loss: 0.5634623765945435
Epoch: 5, Loss: 0.4743380844593048
Epoch: 6, Loss: 0.5775176286697388
Epoch: 7, Loss: 0.5427300930023193
Epoch: 8, Loss: 0.5289063453674316
Epoch: 9, Loss: 0.5700421333312988
Epoch: 10, Loss: 0.4670093059539795
Epoch: 11, Loss: 0.37780749797821045
Epoch: 12, Loss: 0.456510066986084
Epoch: 13, Loss: 0.43402424454689026
Epoch: 14, Loss: 0.5119271278381348
Epoch: 15, Loss: 0.44085758924484253
Epoch: 16, Loss: 0.4962500333786011
Epoch: 17, Loss: 0.3432738780975342
Epoch: 18, Loss: 0.41890835762023926
Epoch: 19, Loss: 0.36760735511779785
Epoch: 20, Loss: 0.2939673066139221
Epoch: 21, Loss: 0.3127586841583252
Epoch: 22, Loss: 0.3694421350955963
Epoch: 23, Loss: 0.29803088307380676
Epoch: 24, Loss: 0.4167473316192627
Epoch: 25, Loss: 0.30215269327163696
Epoch: 26, Loss: 0.2499956637620926
Epoch: 27, Loss: 0.310768663883209

## 추가로 argparse를 통해 hyperparamter튜닝 조금더 편하게 하기 

In [6]:
import argparse
import torch.optim as optim
import torch

parser = argparse.ArgumentParser(description='Spam Classifier')
parser.add_argument('--num_hidden', type=int, default=10, help='number of hidden units')
parser.add_argument('--num_epochs', type=int, default=50, help='number of epochs')
parser.add_argument('--lr', type=float, default=0.0021, help='learning rate')
args = parser.parse_args("")

import torch.nn as nn
class SimpleNN2(nn.Module):

    def __init__(self, args):
        super().__init__()
        self.linear1 = nn.Linear(57, args.num_hidden)
        self.linear2 = nn.Linear(args.num_hidden, 1)
        self.sigmoid = nn.Sigmoid()

        
    def forward(self, x):
        # x: batch_size x 10
        x = self.linear1(x)
        #question
        # batchsize x 5
        x = self.linear2(x)
        # x: batch_size x 1
        x = self.sigmoid(x)
        # x: batch_size x 1
        return x
    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   
model = SimpleNN2(args)
model=model.to(device)

# Create an optimizer
class Trainer2():

    def __init__(self, model, dataloader, args,device):
        # model, dataloader, optimizer, loss_fn
        self.model = model
        self.dataloader = dataloader
        self.optimizer = optim.Adam(model.parameters(), lr=args.lr)
        self.loss_fn = torch.nn.BCELoss()
        self.device = device
    
    def train(self, num_epochs):
        for epoch in range(num_epochs):
            for (x, y) in self.dataloader:
                x=x.to(self.device)
                y=y.to(self.device)
                y_pred = self.model(x)
                y_pred = y_pred.squeeze()
                loss = self.loss_fn(y_pred, y)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()


            print("Epoch: {}, Loss: {}".format(epoch, loss.item()))

    def test(self,test_dataloader):
        with torch.no_grad():
            self.model.eval()
            for (x, y) in test_dataloader:
                x=x.to(self.device)
                y=y.to(self.device)
                y_pred = self.model(x)
                y_pred = y_pred.squeeze()
                loss = self.loss_fn(y_pred, y)
                print("Test Loss: {}".format(loss.item()))
    


#이런식으로 모델에 args라는 인자를 넣어주어 한꺼번에 사용할 수 있음
# Create a model
#model = SimpleNN2(args)
trainer = Trainer2(model, train_dataloader, args,device)

# Train the model
trainer.train(num_epochs=50)
# Test the model

trainer.test(test_dataloader)

Epoch: 0, Loss: 0.3732531666755676
Epoch: 1, Loss: 0.3206535577774048
Epoch: 2, Loss: 0.49055129289627075
Epoch: 3, Loss: 0.2309817522764206
Epoch: 4, Loss: 0.4363638162612915
Epoch: 5, Loss: 0.4048258364200592
Epoch: 6, Loss: 0.4789782166481018
Epoch: 7, Loss: 0.21330952644348145
Epoch: 8, Loss: 0.41494715213775635
Epoch: 9, Loss: 0.12117276340723038
Epoch: 10, Loss: 0.20489202439785004
Epoch: 11, Loss: 0.21682344377040863
Epoch: 12, Loss: 0.23121750354766846
Epoch: 13, Loss: 0.2408123016357422
Epoch: 14, Loss: 0.09381414204835892
Epoch: 15, Loss: 0.19657880067825317
Epoch: 16, Loss: 0.46432480216026306
Epoch: 17, Loss: 0.26288583874702454
Epoch: 18, Loss: 0.11380885541439056
Epoch: 19, Loss: 0.27657991647720337
Epoch: 20, Loss: 0.23897024989128113
Epoch: 21, Loss: 0.4510171115398407
Epoch: 22, Loss: 0.2768315374851227
Epoch: 23, Loss: 0.19579559564590454
Epoch: 24, Loss: 0.18045809864997864
Epoch: 25, Loss: 0.24488452076911926
Epoch: 26, Loss: 0.24081039428710938
Epoch: 27, Loss: 0.1